In [2]:
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

In [5]:
vector_store = client.beta.vector_stores.create(
    name='Clinical Obstetrics The Fetus & Mother - Questions and Answers',
    
)

files = ['data/Clinical Obstetrics - 2007 - Reece - Questions and Answers Clinical Obstetrics The Fetus   Mother-2.pdf']
file_stream = [open(f, 'rb') for f in files]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=file_stream,
)

In [6]:
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [7]:
assistant = client.beta.assistants.create(
    name='Obstetrics Assistant',
    instructions="You are an expert in obstetrics, providing answers to common questions about pregnancy, fetal development, and motherhood. Your answers should be concise and relevant to the users query.",
    tools=[{'type': 'file_search'}],
    tool_resources={
        'file_search': {
            'vector_store_ids': [vector_store.id],
        }
    },
    model='gpt-4o'
)

In [8]:
thread = client.beta.threads.create()

In [9]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content='What does the Fetus feel during birth?'
)

In [10]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions='The name of the user is Daniel and he is a premium user.'
)

In [11]:
import time

while run.status in ['queued', 'in_progress', 'cancelling']:
    print('Thinking...')
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

Thinking...
Thinking...
Thinking...
Thinking...


In [12]:
if run.status == 'completed':
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    print(messages)
else:
    print(run.status)
    print('Error occurred while running the thread.')

SyncCursorPage[Message](data=[Message(id='msg_0zhe37Pvy373qceAqKzZyZod', assistant_id='asst_CVStD0c01eYt8d3y6SQv3uyG', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=483, file_citation=FileCitation(file_id='file-NYcqCRhoAfK9YMCPYa8Bthll'), start_index=470, text='【4:19†source】', type='file_citation'), FileCitationAnnotation(end_index=760, file_citation=FileCitation(file_id='file-NYcqCRhoAfK9YMCPYa8Bthll'), start_index=747, text='【4:15†source】', type='file_citation'), FileCitationAnnotation(end_index=976, file_citation=FileCitation(file_id='file-NYcqCRhoAfK9YMCPYa8Bthll'), start_index=964, text='【4:9†source】', type='file_citation'), FileCitationAnnotation(end_index=1354, file_citation=FileCitation(file_id='file-NYcqCRhoAfK9YMCPYa8Bthll'), start_index=1341, text='【4:19†source】', type='file_citation'), FileCitationAnnotation(end_index=1367, file_citation=FileCitation(file_id='file-NYcqCRhoAfK9YMCPYa8Bthll'), start_index=

In [13]:
print(messages.data[0].content[0].text.value)

While the specifics of what a fetus feels during birth aren't fully comprehended, there is notable insight into the physiological and biochemical responses experienced by the fetus:

1. **Fetal Response to Hypoxemia and Stress**:
    - During labor, fetal hypoxemia can alter the shape of the fetal ECG waveform, specifically the ST segment, due to metabolic acidemia. This suggests that the fetus can experience stress and oxygen deprivation during the birthing process【4:19†source】.

2. **Cardiovascular Adaptations**:
    - The fetal cardiovascular system responds to labor with increased blood flow resistance in response to various conditions. These measures are crucial to monitoring and ensuring fetal well-being during the birthing process【4:15†source】.

3. **Hormonal Influence**:
    - Fetal cortisol production increases, which may play a role in the initiation of labor, contributing to the maturation of fetal organs and preparing the fetus for birth 【4:9†source】.

These observations de